In [42]:
import pandas as pd

In [43]:
df=pd.read_csv("train.csv")
df=df.dropna()

In [44]:
x=df.drop('label',axis=1)
y=df['label']

In [45]:
print(x.shape)
y.shape

(18285, 4)


(18285,)

In [46]:
import tensorflow as tf

In [92]:
import tensorflow 
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM ,Dropout ,Bidirectional
from tensorflow.keras.layers import Dense

In [48]:
message=x.copy()

In [49]:
import nltk
import re
from nltk.corpus import stopwords

In [50]:
message.reset_index(inplace=True)

In [51]:

from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(message)):
    
    review = re.sub('[^a-zA-Z]', ' ', message['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [52]:
voc_size=5000
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[4536, 531, 4615, 3038, 2359, 3575, 620, 3273, 3558, 507],
 [823, 3540, 4354, 1631, 141, 4000, 808],
 [3168, 3743, 1234, 2950],
 [3902, 4475, 2708, 3425, 3768, 4178],
 [1277, 141, 1254, 16, 2685, 4677, 141, 2919, 234, 3968],
 [465,
  2619,
  347,
  2755,
  1549,
  4509,
  318,
  3989,
  1393,
  3391,
  1245,
  2243,
  3175,
  2917,
  808],
 [1946, 4723, 3317, 2129, 4, 571, 2839, 2714, 4246, 441, 2083],
 [2610, 726, 2436, 2852, 133, 73, 4509, 1532, 4246, 441, 2083],
 [3565, 2729, 2923, 2062, 459, 2113, 1613, 416, 4509, 2173],
 [4412, 3689, 837, 3920, 2381, 4621, 3716, 732],
 [4615, 4715, 114, 529, 2474, 171, 924, 799, 3507, 1180, 4908],
 [3425, 3069, 2359, 2113, 4509, 133],
 [3209, 4545, 1790, 1106, 755, 2216, 3331, 3557, 4136],
 [962, 3391, 4185, 338, 3139, 3833, 457, 4246, 441, 2083],
 [2711, 4118, 985, 4349, 1531, 4246, 441, 2083],
 [3887, 1740, 758, 3911, 2034, 4610, 2581, 695, 220, 2574],
 [526, 3192, 3540],
 [4857, 4413, 94, 1014, 4509, 996, 3589, 808],
 [3961, 508, 4354, 493, 28

In [53]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 3273 3558  507]
 [   0    0    0 ...  141 4000  808]
 [   0    0    0 ... 3743 1234 2950]
 ...
 [   0    0    0 ... 4246  441 2083]
 [   0    0    0 ...   27 2072 4081]
 [   0    0    0 ... 4362 2399 2447]]


In [54]:
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [55]:
len(embedded_docs),y.shape

(18285, (18285,))

In [56]:
import numpy as np
X_final=np.array(embedded_docs)
Y_final=np.array(y)

In [57]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(X_final,Y_final,test_size=0.33,random_state=42)

In [58]:
model.fit(X_train,Y_train,validation_data=(X_test,Y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 65s 340ms/step - loss: 0.3358 - accuracy: 0.8374 - val_loss: 0.1972 - val_accuracy: 0.9143
Epoch 2/10
192/192 [==============================] - 69s 359ms/step - loss: 0.1407 - accuracy: 0.9436 - val_loss: 0.1944 - val_accuracy: 0.9191
Epoch 3/10
192/192 [==============================] - 63s 329ms/step - loss: 0.0999 - accuracy: 0.9622 - val_loss: 0.2292 - val_accuracy: 0.9198
Epoch 4/10
192/192 [==============================] - 63s 331ms/step - loss: 0.0748 - accuracy: 0.9731 - val_loss: 0.2671 - val_accuracy: 0.9132
Epoch 5/10
192/192 [==============================] - 63s 328ms/step - loss: 0.0515 - accuracy: 0.9817 - val_loss: 0.3287 - val_accuracy: 0.9127
Epoch 6/10
192/192 [==============================] - 64s 333ms/step - loss: 0.0357 - accuracy: 0.9889 - val_loss: 0.3792 - val_accuracy: 0.9142
Epoch 7/10
192/192 [==============================] - 63s 326ms/step - loss: 0.0210 - accuracy: 0.9936 - val_loss: 0.4381 - val_ac

In [88]:
y_predict=model.predict_classes(X_test)
from sklearn.metrics import confusion_matrix,classification_report

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [89]:
confusion_matrix(Y_test,y_predict)

array([[3146,  273],
       [ 269, 2347]], dtype=int64)

In [90]:
print(classification_report(Y_test,y_predict))

              precision    recall  f1-score   support

           0       0.92      0.92      0.92      3419
           1       0.90      0.90      0.90      2616

    accuracy                           0.91      6035
   macro avg       0.91      0.91      0.91      6035
weighted avg       0.91      0.91      0.91      6035



In [93]:
embedding_vector_features=40
model1=Sequential()
model1.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model1.add(Bidirectional(LSTM(100)))
model1.add(Dropout(0.3))
model1.add(Dense(1,activation='sigmoid'))
model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model1.summary())

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
bidirectional (Bidirectional (None, 200)               112800    
_________________________________________________________________
dropout (Dropout)            (None, 200)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 201       
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________
None


In [94]:
model1.fit(X_train,Y_train,validation_data=(X_test,Y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 135s 705ms/step - loss: 0.3161 - accuracy: 0.8484 - val_loss: 0.2056 - val_accuracy: 0.9148
Epoch 2/10
192/192 [==============================] - 126s 657ms/step - loss: 0.1460 - accuracy: 0.9467 - val_loss: 0.2124 - val_accuracy: 0.9180
Epoch 3/10
192/192 [==============================] - 119s 619ms/step - loss: 0.1020 - accuracy: 0.9622 - val_loss: 0.2372 - val_accuracy: 0.9107
Epoch 4/10
192/192 [==============================] - 116s 606ms/step - loss: 0.0735 - accuracy: 0.9736 - val_loss: 0.2515 - val_accuracy: 0.9133
Epoch 5/10
192/192 [==============================] - 116s 604ms/step - loss: 0.0501 - accuracy: 0.9837 - val_loss: 0.3364 - val_accuracy: 0.9142
Epoch 6/10
192/192 [==============================] - 111s 577ms/step - loss: 0.0315 - accuracy: 0.9897 - val_loss: 0.3437 - val_accuracy: 0.9147
Epoch 7/10
192/192 [==============================] - 110s 574ms/step - loss: 0.0218 - accuracy: 0.9926 - val_loss: 0.4220 -

In [95]:
y_predict=model.predict_classes(X_test)
confusion_matrix(Y_test,y_predict)

array([[3146,  273],
       [ 269, 2347]], dtype=int64)

In [97]:
print(classification_report(Y_test,y_predict))

              precision    recall  f1-score   support

           0       0.92      0.92      0.92      3419
           1       0.90      0.90      0.90      2616

    accuracy                           0.91      6035
   macro avg       0.91      0.91      0.91      6035
weighted avg       0.91      0.91      0.91      6035

